## CODIGO


# Model Design

In [472]:
# Model design
#pip install agentpy
import agentpy as ap
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import IPython

# Aux
import random

## Model Definition

In [473]:
class VehicleAgent(ap.Agent):

  """ An agent tha simulates a vehicle """

  def setup(self):
    self.map = self.model.map
    self.random = self.model.random
    self.xUbication = 0
    self.yUbication = 0
    self.fuel = 0
    self.status = "looking"
    self.stepsParking = 0

  def moveUp(self):
    self.map.move_by(self, (-1, 0))

  def park(self):
    self.status = 'parked'

  def findParking(self):

    # Obtenemos estacionamientos libres en el grid
    pAgents = [i for i in self.model.map.agents if 'ParkingAgent' in i.type and i.availability]
    
    for parkA in pAgents:
      pass
      #print(parkA.type)

  def movement(self):

    # Obtenemos los vecinos al vehiculo
    neighbors = self.map.neighbors(self, distance = 0)
    roadNeighbors = [i for i in neighbors if 'VehicleAgent' not in i.type and 'ParkingAgent' not in i.type]

    # Observamos los posibles movimientos <- Aqui logica de mejor mov al estacionamiento
    for road in roadNeighbors:
      print(road.direction)
      
      # Camino hacia la arriba
      if(road.direction == 'North'):
        self.map.move_by(self, (-1, 0))

      elif(road.direction == 'South'):
        self.map.move_by(self, (1, 0))

      elif(road.direction == 'West'):
        self.map.move_by(self, (0, -1))
      
      else:
        self.map.move_by(self, (0, 1))
      
    
    # Generamos el movimiento


In [474]:
class ParkingAgent(ap.Agent):

  """ An agent that simulates a parking slot """

  def setup(self):
    self.map = self.model.map
    self.random = self.model.random
    self.id
    self.availability = True
    self.xUbication = 0
    self.yUbication = 0

  def setAvailability(self):
    # Condicional en dependencia a los agentes vehiculo
    pass


      


In [475]:
def randomPos(n):
    positions = [(0,4), (11,0), (15, 11), (4, 15)]
    return random.sample(positions, n)


In [476]:
class CongestionModel(ap.Model):

  """ A simple model of random wealth transfers """

  def setup(self):

    # Creacion de Grid
    self.map = ap.Grid(self, (16, 16), track_empty = True)

    # Creacion de Agentes
    self.vehicleAgents = []
    self.parkingAgents = ap.AgentList(self, self.p.parkingAgents, ParkingAgent)
    parkPositions = [(6,10), (10, 9), (10, 14)]

    # Agente camino
    # Camino hacia arriba
    road = []
    pos = []

    for i in range(0,4,1):
      
      temp = ap.AgentList(self, 16)
      
      if (i == 0):
        temp.direction = "East"
        for j in range(0,16,1):
          pos += [(11,j)]
      elif (i == 1):
        temp.direction = "West"
        for j in range(0,16,1):
          pos += [(4,j)]
      elif (i == 2):
        temp.direction = "South"
        for j in range(0,16,1):
          pos += [(j,4)]
      else:
        temp.direction = "North"
        for j in range(0,16,1):
          pos += [(j,11)]
      
      road += temp
    
    self.roads = road
    

    #self.map.add_agents(self.vehicleAgents, random=True, empty=True)
    
    self.map.add_agents(self.parkingAgents, positions = parkPositions)
    self.map.add_agents(self.roads, positions = pos)

    #self.map.move_by(self.parkingAgents.random(), (3,3))

    #self.vehicleAgents[0].findParking()

    """ self.map.move_by(self.vehicleAgents.random(), (3,3))
    self.map.move_by(self.parkingAgents.random(), (3,3)) """
 
  def update(self):
    a = 2
    #self.record('Traffic', traffic())

  def step(self):
    numCars = random.randint(0,3)
    positions = randomPos(numCars)

    newCars = ap.AgentList(self, numCars, VehicleAgent)

    self.vehicleAgents += newCars

    if(len(positions) > 0):
      self.map.add_agents(newCars, positions = (positions))

    vehAgents = [i for i in self.map.agents if 'VehicleAgent' in i.type]

    for vehicle in vehAgents:
      vehicle.movement()

    #print(positions)


  def end(self):
    pass
    #self.agents.record('')

## Simulation Run

In [477]:
parameters = {
  'vehicleAgents': 10,
  'parkingAgents': 3,
  'steps': 5,
  'seed': 42,
}

model = CongestionModel(parameters)
#results = model.run()
#results

In [471]:
# Create single-run animation with custom colors

def view_agents(Agent):
  num = 0
  
  if('VehicleAgent' in Agent.type):
    num = 0
  elif('ParkingAgent' in Agent.type):
    num = 1
  elif('Agent' in Agent.type):
    num = 2
  else:
    num = 3

  return num

def animation_plot(model, ax):
    attr_grid = model.map.apply(view_agents)
    #print(attr_grid)
    #attr_grid = model.map.attr_grid(isinstance())
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#848484', 3:'#e5e5e5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    #ax.set_title(f"Simulation of a forest fire\n"
                 #f"Time-step: {model.t}, Trees left: "
                 #f"{len(model.agents.select(model.agents.condition == 0))}")

fig, ax = plt.subplots()
model = CongestionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

East
East
South
South
East
East
East
South
South
South
South
South
East
East
East
South
West
West
South
West
South
South
West
West
North
North
South
South
